In [ ]:
from google.colab import drive

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from scipy.interpolate import interp1d


In [ ]:
drive.mount('/content/drive', force_remount=True)
data_dir = "/content/drive/My Drive/Inzynierka/Dane/"

Mounted at /content/drive


In [ ]:
competition = "accelerometer-biometric-competition.zip"
csv_train = data_dir+"Competition/train.csv"

In [ ]:
columns = ['X','Y','Z','Device']
df_train = pd.read_csv(csv_train, usecols=columns)
# scaler = StandardScaler()
# df_train_features = scaler.fit_transform(df_train.drop('Device', axis=1))
# df_train[['X', 'Y', 'Z']] = df_train_features

In [ ]:
device_counts = df_train['Device'].value_counts()
smallest = device_counts.nsmallest(30)
print(smallest)

Device
263     3071
211     3122
987     3133
1017    3202
412     3259
1031    3271
213     3274
73      3317
998     3392
823     3403
642     3433
612     3437
335     3507
836     3539
871     3617
776     3620
638     3661
1029    3718
713     3727
683     3767
933     3793
805     3890
700     3920
423     3946
343     4013
447     4031
706     4099
149     4109
580     4159
600     4182
Name: count, dtype: int64


In [ ]:
with open('/content/drive/MyDrive/Inzynierka/Dane/moje/iga3020241122123434.txt', 'r') as file:
    lines = file.readlines()

# Konwersja danych do tablicy floatów
my_data = []
for line in lines:
    row = list(map(float, line.strip().split(',')))
    my_data.append(row)

# Wyświetlenie wyniku
my_data = np.array(my_data)
my_data.shape

(1512, 3)

#Tworzenie sekwencji

In [ ]:
owner_id = 0
seq_length = 30
n = 10 #liczba próbek klasy 1
m = 2 #liczba próbek klasy 0 z każdego device

sequences_count = []
# data = []

device_unique = df_train.Device.unique()
data = pd.DataFrame({'X': [], 'Y': [],'Z': [], 'Device': []})
data_val = pd.DataFrame({'X': [], 'Y': [],'Z': [], 'Device': []})
data_test = pd.DataFrame({'X': [], 'Y': [],'Z': [], 'Device': []})
for device in device_unique:
  index = df_train.loc[df_train['Device'] == device].index[0]
  # n = device_counts[device] // 300
  if device == owner_id:
    data = pd.concat([data, df_train.iloc[index:index+n*seq_length]], ignore_index=True)
    sequences_count.append(n)
    data_val = pd.concat([data_val, df_train.iloc[index+n*seq_length:index+(n+2)*seq_length]], ignore_index=True)
    data_test = pd.concat([data_test, df_train.iloc[index+(2+n)*seq_length:index+(n+4)*seq_length]], ignore_index=True)
  else:
    data = pd.concat([data, df_train.iloc[index:index+m*seq_length]], ignore_index=True)
    sequences_count.append(m)

In [ ]:
data_val['Device'] = 1
data_test['Device'] = 1

In [ ]:
def create_sequences(data, seq_length):
    X = []
    for i in range (int(len(data) / seq_length)):
        X.append(data[i:i+seq_length])
    return np.array(X)

In [ ]:
def prepare_my_data(data, seq_length):
    X = []
    n = data.shape[0]//seq_length
    return np.array(data[:n*seq_length]), np.ones(n)

In [ ]:
X = data[['X', 'Y', 'Z']].to_numpy()
y = data['Device'].to_numpy()
# my_X, my_y = prepare_my_data(my_data, seq_length)
# X = np.vstack((X, my_X))

X_seq= create_sequences(X, seq_length)
y_seq = y[::seq_length]
y_seq = np.where(y_seq == owner_id, 1, 0)
# y_seq = np.hstack((y_seq, my_y))
print(X_seq.shape)
print(y_seq.shape)

(774, 30, 3)
(774,)


In [ ]:
my_X, my_y = prepare_my_data(my_data, seq_length)
my_X_seq = create_sequences(my_X, seq_length)

#Interpolacja

In [ ]:
def interpolate_data(data):
  n_samples_5 = data.shape[0]  # Liczba próbek
  indices_5 = np.arange(n_samples_5)  # Indeksy próbek [0, 1, ..., n_samples_5-1]
  indices_50 = np.linspace(0, n_samples_5 - 1, (n_samples_5 - 1) * 10 + 1)  # Indeksy dla 50 Hz

  # Interpolacja każdej kolumny osobno
  interpolated_data = np.zeros((len(indices_50), data.shape[1]))  # Przygotowanie tablicy wynikowej

  for i in range(data.shape[1]):  # Iteracja po wymiarach (kolumnach)
      interp_func = interp1d(indices_5, data[:, i], kind='quadratic')  # Interpolacja liniowa
      interpolated_data[:, i] = interp_func(indices_50)  # Wynik dla wymiaru i
  return interpolated_data

In [ ]:
X_seq_interpolated = []
for seq in X_seq:
  interpolated_data = interpolate_data(seq)
  X_seq_interpolated.append(interpolated_data)
X_seq_interpolated=np.array(X_seq_interpolated)

In [ ]:
X_seq_interpolated.shape

(774, 291, 3)

In [ ]:
my_X, my_y = prepare_my_data(my_data, X_seq_interpolated.shape[1])
my_X_seq = create_sequences(my_X, X_seq_interpolated.shape[1])
my_X_seq.shape

(5, 291, 3)

In [ ]:
X_seq_all = np.concatenate((X_seq_interpolated, my_X_seq))
y_seq = np.hstack((y_seq, my_y))
X_seq_all.shape

(779, 291, 3)

In [ ]:
reshaped_data = X_seq_all.reshape(-1, 3)

print("Nowy kształt:", reshaped_data.shape)

Nowy kształt: (226689, 3)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(reshaped_data)

In [ ]:
original_shape = (779, 291, 3)
X_seq_all = scaled_data.reshape(original_shape)

print("Przywrócony kształt:", X_seq_all.shape)

Przywrócony kształt: (779, 291, 3)


In [ ]:
X_seq_all

array([[[-0.11615106,  1.06641277, -0.50875901],
        [-0.11109988,  1.06837548, -0.5008188 ],
        [-0.10677226,  1.07041007, -0.49407037],
        ...,
        [-0.22837075,  1.07838922, -0.41384356],
        [-0.22340698,  1.07829332, -0.41155722],
        [-0.21787985,  1.07784344, -0.40889376]],

       [[-0.09819892,  1.08927438, -0.48298726],
        [-0.10173678,  1.09344571, -0.49497463],
        [-0.10555229,  1.09719811, -0.50551513],
        ...,
        [-0.2143023 ,  1.07035572, -0.4393235 ],
        [-0.21586747,  1.0684878 , -0.44630561],
        [-0.21787985,  1.06641277, -0.45399402]],

       [[-0.14607129,  1.11213627, -0.53775201],
        [-0.15690003,  1.11054327, -0.52998667],
        [-0.1665192 ,  1.10905024, -0.5225152 ],
        ...,
        [-0.27766294,  1.09735048, -0.40230973],
        [-0.28920733,  1.10439081, -0.38868021],
        [-0.3016565 ,  1.11213627, -0.37345757]],

       ...,

       [[ 0.05402024, -0.37175951,  0.68930572],
        [-0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_seq_all,y_seq ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_seq,y_seq ,
#                                    random_state=104,
#                                    test_size=0.25,
#                                    shuffle=True)

In [ ]:
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Sprawdzenie kształtu
print(X_train.shape)

(584, 291, 3)


In [ ]:
class_count=[0,0]

for i in y_train:
  if i == 0:
    class_count[0] += 1
  else:
    class_count[1] += 1

In [ ]:
c = (class_count[0] + class_count[1])/2
class_weight = {0: c/class_count[0], 1: c/class_count[1]}

In [ ]:
class_weight

{0: 0.503448275862069, 1: 73.0}

#Model

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':

  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
model = Sequential()
model.add(LSTM(units=5, recurrent_activation='sigmoid', return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2]), unroll=True))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
  model.fit(X_train, y_train, epochs=10, batch_size=16, class_weight=class_weight, verbose=1)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.8035 - loss: 0.7297
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9973 - loss: 0.3691
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9922 - loss: 0.5610
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9819 - loss: 0.7358
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9559 - loss: 0.2751
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9930 - loss: 0.1155
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9963 - loss: 0.1436
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9961 - loss: 0.1308
Epoch 9/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9827 - loss: 0.1240
Epoch 10/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9839 - loss: 0.0661


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=16, class_weight=class_weight, verbose=1)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy  )

Test loss: 0.019563963636755943
Test accuracy: 0.9948717951774597


In [ ]:
model.export("/content/my_model")
converter = tf.lite.TFLiteConverter.from_saved_model("/content/my_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS
]
converter.experimental_new_converter = False  # Użycie starego konwertera
tflite_float_model = converter.convert()

Saved artifact at '/content/my_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 291, 3), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136211802327344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211674232640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211674232464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673501360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673499072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673506640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673507520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673512800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673513504: TensorSpec(shape=(), dtype=tf.resource, name=None)


Instructions for updating:
Use `tf.saved_model.load` instead.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


In [ ]:
f = open('model.tflite', "wb")
f.write(tflite_float_model)
f.close()

# Download the digit classification model
from google.colab import files
files.download('model.tflite')

print('`model.tflite` has been downloaded')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

`model.tflite` has been downloaded


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS
]
converter.experimental_new_converter = False  # Użycie starego konwertera
tflite_float_model = converter.convert()

Saved artifact at '/tmp/tmpo9un936d'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 291, 3), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136211802327344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211674232640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211674232464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673501360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673499072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673506640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673507520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673512800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136211673513504: TensorSpec(shape=(), dtype=tf.resource, name=None)


AttributeError: 'Sequential' object has no attribute '_get_save_spec'

In [ ]:
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

In [ ]:
# Re-convert the model to TF Lite using quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# Show model size in KBs.
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

Saved artifact at '/tmp/tmpc8mxbgg4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 291, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138203937809232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138203626626512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138207267819712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138203614813472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138203614813296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138203614821040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138203568965408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138203568970688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138203568971568: TensorSpec(shape=(), dtype=tf.resource, name=None)
Quantized model size = 788KBs,


NameError: name 'float_model_size' is not defined

In [ ]:
f = open('model.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

# Download the digit classification model
from google.colab import files
files.download('model.tflite')

print('`model.tflite` has been downloaded')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

`model.tflite` has been downloaded


In [ ]:

interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

print("TensorFlow Lite Version:", tf.__version__)
print("Required Ops:")

for op in interpreter._get_ops_details():
    print(op)

TensorFlow Lite Version: 2.17.1
Required Ops:
{'index': 0, 'op_name': 'SHAPE', 'inputs': array([0], dtype=int32), 'outputs': array([16], dtype=int32), 'operand_types': [<class 'numpy.float32'>], 'result_types': [<class 'numpy.int32'>]}
{'index': 1, 'op_name': 'STRIDED_SLICE', 'inputs': array([16,  2,  3,  3], dtype=int32), 'outputs': array([17], dtype=int32), 'operand_types': [<class 'numpy.int32'>, <class 'numpy.int32'>, <class 'numpy.int32'>, <class 'numpy.int32'>], 'result_types': [<class 'numpy.int32'>]}
{'index': 2, 'op_name': 'TRANSPOSE', 'inputs': array([0, 4], dtype=int32), 'outputs': array([18], dtype=int32), 'operand_types': [<class 'numpy.float32'>, <class 'numpy.int32'>], 'result_types': [<class 'numpy.float32'>]}
{'index': 3, 'op_name': 'UNPACK', 'inputs': array([18], dtype=int32), 'outputs': array([ 19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  5

In [ ]:
# import numpy as np
# from scipy.interpolate import interp1d

# # Sygnał 5 Hz
# t_5 = np.arange(0, 2, 0.2)  # Próbki co 0.2 sekundy
# x_5 = np.sin(2 * np.pi * t_5)  # Przykładowe wartości sygnału

# # Sygnał 50 Hz (dostosowanie zakresu czasu)
# t_50 = np.arange(0, t_5[-1] + 0.02, 0.02)  # Próbki co 0.02 sekundy, zgodnie z zakresem t_5

# # Interpolacja liniowa
# interp_func = interp1d(t_5, x_5, kind='linear')
# x_50 = interp_func(t_50)  # Interpolowany sygnał
# print(t_5)
# print(x_5)
# print(x_50)

In [ ]:


data = X_seq[0]
n_samples = len(data)
time = np.linspace(0, (n_samples - 1) * 0.2, n_samples)

# Wykres 2D: każda zmienna względem czasu
plt.figure(figsize=(10, 6))
plt.plot(time, data[:, 0], label='X', marker='o')
# plt.plot(time, data[:, 1], label='Y', marker='s')
plt.plot(time, data[:, 2], label='Z', marker='^')
plt.xlabel('Czas [s]')
plt.ylabel('Wartość')
plt.title('Dane w czasie')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from scipy.interpolate import interp1d


# Sygnał próbkowany z 5 Hz (indeksy jako "czas")
x_5 = data  # Wartości sygnału
n_samples_5 = len(x_5)

# Indeksy próbek
indices_5 = np.arange(n_samples_5)  # [0, 1, 2, 3, 4]

# Interpolowane indeksy dla 50 Hz
indices_50 = np.linspace(0, n_samples_5 - 1, (n_samples_5 - 1) * 10 + 1)

# Interpolacja
interp_func = interp1d(indices_5, x_5, kind='linear')
x_50 = interp_func(indices_50)

print("Interpolowany sygnał (50 Hz):", x_50)

# interp_func = interp1d(time, data, kind='quadratic')  # Wielomian drugiego stopnia
# Wykres 2D: każda zmienna względem czasu
plt.figure(figsize=(10, 6))
plt.plot(indices_50, x_50[:, 0], label='X', marker='o')
# plt.plot(time, data[:, 1], label='Y', marker='s')
plt.plot(indices_50, x_50[:, 2], label='Z', marker='^')
plt.xlabel('Czas [s]')
plt.ylabel('Wartość')
plt.title('Dane w czasie')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import numpy as np


def interpolate_data(data):
  n_samples_5 = data.shape[0]  # Liczba próbek
  indices_5 = np.arange(n_samples_5)  # Indeksy próbek [0, 1, ..., n_samples_5-1]
  indices_50 = np.linspace(0, n_samples_5 - 1, (n_samples_5 - 1) * 10 + 1)  # Indeksy dla 50 Hz

  # Interpolacja każdej kolumny osobno
  interpolated_data = np.zeros((len(indices_50), data.shape[1]))  # Przygotowanie tablicy wynikowej

  for i in range(data.shape[1]):  # Iteracja po wymiarach (kolumnach)
      interp_func = interp1d(indices_5, data[:, i], kind='quadratic')  # Interpolacja liniowa
      interpolated_data[:, i] = interp_func(indices_50)  # Wynik dla wymiaru i
  return interpolated_data



In [ ]:
X_seq_interpolated = []
for seq in X_seq:
  interpolated_data = interpolate_data(seq)
  X_seq_interpolated.append(interpolated_data)

In [ ]:
X_seq_interpolated = np.array(X_seq_interpolated)
X_seq_interpolated.shape

In [ ]:


plt.figure(figsize=(10, 6))
plt.plot(indices_50 * 0.2, X_seq_interpolated[5][:, 0], label='X', marker='.')
# plt.plot(indices_50 * 0.2, interpolated_data[:, 1], label='Y', marker='s')
plt.plot(indices_50 * 0.2, X_seq_interpolated[5][:, 2], label='Z', marker='.')
plt.xlabel('Czas [s]')
plt.ylabel('Wartość')
plt.title('Interpolowane dane w czasie (50 Hz)')
plt.legend()
plt.grid()
plt.show()